# 第三题：支持向量机的分类任务

实验内容：
1. 使用支持向量机完成Breast_Cancer_Wisconsin数据集分类任务
2. 使用不同核函数和惩罚系数C在训练集上训练模型，并分别计算对应模型在测试集的精度，查准率，查全率，F1值，将结果填写到下表中。

| 核函数 | C | 精度 | 查准率 | 查全率 | F1| 
| - | - | - | - | - | - |
rbf | 0.1 |  |  |  | 
rbf | 1 |  |  | | 
linear | 0.1 |  |  |  | 
linear | 1 |  |  |  | 
sigmoid | 0.1 |  |  | | 
sigmoid | 1 |  |  |  | 

In [1]:
# 导入数据
import numpy as np
import pandas as pd
data = pd.read_csv('data\Breast_Cancer_Wisconsin\data')
data['diagnosis'] = data['diagnosis'].apply(lambda x:1 if x == "M" else 0)

In [2]:
# 查看数据类型
data.dtypes

id                           int64
diagnosis                    int64
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst     

In [3]:
# 划分数据为特征和标签
data = data.values
x = data[:,2:-1]
y = data[:,1:2].reshape(-1)

In [4]:
x

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [5]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0.

In [6]:
# 数据集分割
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.3, random_state = 32)
trainX.shape, trainY.shape, testX.shape, testY.shape

((398, 30), (398,), (171, 30), (171,))

**注意：计算线性核的时候，要使用 LinearSVC 这个类，不要使用SVC(kernel = 'linear')。LinearSVC不需要设置kernel参数！**

In [7]:
# 引入模型
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [9]:
# YOUR CODE HERE
# 定义一个函数，用于训练SVM模型并评估性能
def train_and_evaluate_svm(kernel, C, trainX, trainY, testX, testY):
    # 初始化SVM模型
    if kernel == 'linear':
        model = SVC(kernel=kernel, C=C)
    else:
        model = SVC(kernel=kernel, C=C, gamma='auto')  # For non-linear kernels, use gamma='auto'
    
    # 在训练集上训练模型
    model.fit(trainX, trainY)
    
    # 在训练集和测试集上进行预测
    train_preds = model.predict(trainX)
    test_preds = model.predict(testX)
    
    # 计算性能指标
    accuracy = accuracy_score(testY, test_preds)
    precision = precision_score(testY, test_preds)
    recall = recall_score(testY, test_preds)
    f1 = f1_score(testY, test_preds)
    
    return accuracy, precision, recall, f1

In [10]:
# 定义不同的核函数和惩罚系数C的组合
kernels = ['linear', 'rbf', 'sigmoid']
C_values = [0.1, 1, 10]

# 遍历不同的核函数和C值，训练模型并输出性能指标
for kernel in kernels:
    for C in C_values:
        accuracy, precision, recall, f1 = train_and_evaluate_svm(kernel, C, trainX, trainY, testX, testY)
        print(f"Kernel: {kernel}, C: {C}")
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        print("-" * 40)

Kernel: linear, C: 0.1
Accuracy: 0.9064, Precision: 0.9032, Recall: 0.8485, F1: 0.8750
----------------------------------------
Kernel: linear, C: 1
Accuracy: 0.9298, Precision: 0.9355, Recall: 0.8788, F1: 0.9062
----------------------------------------
Kernel: linear, C: 10
Accuracy: 0.9298, Precision: 0.9219, Recall: 0.8939, F1: 0.9077
----------------------------------------
Kernel: rbf, C: 0.1
Accuracy: 0.6140, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
----------------------------------------
Kernel: rbf, C: 1
Accuracy: 0.6140, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
----------------------------------------
Kernel: rbf, C: 10
Accuracy: 0.6140, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
----------------------------------------
Kernel: sigmoid, C: 0.1
Accuracy: 0.6140, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
----------------------------------------
Kernel: sigmoid, C: 1
Accuracy: 0.6140, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
--------------------------------

C:\Users\苍山沐雪\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\苍山沐雪\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\苍山沐雪\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\苍山沐雪\AppData\Local\Pro